In [1]:
# !pip install opendatasets

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
# import opendatasets as od

In [3]:
# od.download("https://www.kaggle.com/datasets/kazanova/sentiment140")

Skipping, found downloaded files in ".\sentiment140" (use force=True to force download)


In [4]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Dubai
[nltk_data]     Computers\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [8]:
twitter_data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin1', on_bad_lines="skip")

In [9]:
twitter_data.shape

(1599999, 6)

In [10]:
# Define the custom column names to assign to each column in the dataset
# The original CSV has no header, so we provide our own
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']

# Load the Sentiment140 CSV dataset using pandas
twitter_data = pd.read_csv(
    "training.1600000.processed.noemoticon.csv",  # Path to the CSV file (1.6 million tweets)
    names=column_names,  # Assign the custom column names we defined above (since original file has no header row)
    encoding='latin1',  # Encoding used to properly read special characters in tweets (like emojis or symbols)
    engine='c',  # Use the C parser (default and faster), handles large files more efficiently than 'python' engine
    on_bad_lines="skip"  # If a row is malformed (e.g., missing or extra columns), skip that row instead of throwing an error
)


In [11]:
twitter_data.shape

(1600000, 6)

In [15]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [17]:
twitter_data["target"].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert 4 to 1

In [18]:
twitter_data.replace({"target": {4:1}}, inplace=True)

0 --> Negative
1 --> Positive

# Stemming

Stemming is process of reducing of a word to its Root word

In [19]:
port_stem = PorterStemmer()

In [20]:
stop_words = stopwords.words("english")

def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]", " ", content) ## ^ mean remove all things excepte character (a-zA-Z) # Remove everything except letters
    stemmed_content = stemmed_content.lower()# Convert to lowercase
    stemmed_content = stemmed_content.split()# Tokenize into words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stop_words]

    return " ".join(stemmed_content)  # Join back to string

In [21]:
twitter_data["stemmed_content"] = twitter_data["text"].apply(stemming)

In [22]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [23]:
print(twitter_data["stemmed_content"])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
print(twitter_data["target"])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


# Separating Label and data

In [25]:
x = twitter_data["stemmed_content"].values
y = twitter_data["target"]

In [26]:
# print(x)

In [27]:
# print(y)

# Train Test Split

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [29]:
print(x.shape, x_train.shape, x_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
# print(x_train)

In [31]:
# print(x_test)

# converting Textual to Numerical

In [32]:
# apply vectorizer to convert in vector matrix
vectorizer = TfidfVectorizer()

In [33]:
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)


Original Tweet	TF-IDF Vector...
"i love ai"	[0.56, 0.78, 0.34, ..., 0],
"ai is amazing"	[0.12, 0.44, 0.89, ..., 0]

# Save Vectorizer Model

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

filename = "vectorizer_jani_model.pkl"
# Save using Pickle
with open("vectorizer_jani_model.pkl", "wb") as file:
    pickle.dump(vectorizer, file)

In [35]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import joblib
# # Save using Joblib
# joblib.dump(vectorizer, "vectorizer_jani_model.pkl")

In [36]:
# print(x_train)

# Training MOdel

In [37]:
lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')

In [38]:
lr_model.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [39]:
lr_model.score(x_test, y_test)*100

77.8646875

# Accuracy

In [40]:
x_train_pred = lr_model.predict(x_train)
train_data_accuracy = accuracy_score(x_train_pred, y_train)

In [41]:
print("Accuracy Score of Training Data : ", train_data_accuracy)

Accuracy Score of Training Data :  0.81027734375


In [42]:
x_test_pred = lr_model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_pred, y_test)

In [43]:
print("Accuracy Score of Training Data : ", test_data_accuracy)

Accuracy Score of Training Data :  0.778646875


# Save the Model 

# Pickle Model

In [49]:
import pickle

# Save model
with open('my_trained_pickle_model.pkl', 'wb') as file:
    pickle.dump(lr_model, file)

In [50]:
# Save vectorizer (optional but recommended)
with open('vectorizer_pickle.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

# Joblib Model

In [51]:
import joblib

# Save model
joblib.dump(lr_model, 'my_Trained_joblib_model.joblib')


['my_Trained_joblib_model.joblib']

In [52]:
# Save vectorizer
joblib.dump(vectorizer, 'vectorizer_joblib.joblib')

['vectorizer_joblib.joblib']

In [53]:
with open('my_trained_pickle_model.pkl', 'rb') as file:
    loaded_model_pickle = pickle.load(file)

with open('vectorizer_pickle.pkl', 'rb') as file:
    loaded_vectorizer_pickle = pickle.load(file)

In [54]:
x_train_pred = loaded_model_pickle.predict(x_train)
train_data_accuracy = accuracy_score(x_train_pred, y_train)

In [55]:
train_data_accuracy

0.81027734375

#  Predictionsss

In [63]:
custom_tweets = [
    "I hate this app, it's terrible!",
    "This is amazing, I love it!",
    "Worst experience ever.",
    "I'm so happy with the service."
]

# Transform and predict
transformed = loaded_vectorizer_pickle.transform(custom_tweets)
predictions = loaded_model_pickle.predict(transformed)

# Show results
for tweet, pred in zip(custom_tweets, predictions):
    sentiment = "Positive " if pred == 1 else "Negative"
    print(f"Tweet: {tweet}\nPrediction: {sentiment}\n")


Tweet: I hate this app, it's terrible!
Prediction: Negative

Tweet: This is amazing, I love it!
Prediction: Positive 

Tweet: Worst experience ever.
Prediction: Negative

Tweet: I'm so happy with the service.
Prediction: Positive 



In [ ]:
# # Load the tweet
# x_new = x_test[300]  # Raw tweet text

# # Transform the tweet using the same vectorizer used in training
# x_new_transformed = vectorizer.transform([x_new])  # Convert to feature vector

# # Print original sentiment
# print("Original Sentiment:", y_test.iloc[300])

# # Make prediction
# prediction = loaded_model.predict(x_new_transformed)
# print("Prediction:", prediction)

# if prediction[0] == 0:
#     print("Negative Tweet")
# else:
#     print("Positive Tweet")

In [ ]:
# x_new = x_test[3]
# print("Actual Sentiment:", y_test.iloc[3])

# x_new_vector = vectorizer.transform([x_new])
# prediction = loaded_model.predict(x_new_vector)
# print("Prediction:", prediction)

# if (prediction[0] == 0):
#   print("Negative Tweet")
# else:
#   print("Positive Tweet")